# Test: what is the best way to quantify the starburst age?
---

In this notebook:
* come up with a few different ways to quantify the "starburst age".
* write necessary functions to compute these quantities given a star formation history.
* Using MC runs (and later on regularisation too), see whether any of these quantities can reliably be measured.

Ideas:
* As a control: previous method - time index at which most recent star formation event drops to 0.
    * *potential issue*: definitely unreliable, given the "noise" in the recovered SFH that persists at even very high S/N.
* Mass weighted age below some age threshold.
    * *potential issue*: choice of age threshold is somewhat arbitrary.
* Counting backwards from $t = 0$, the time index at which the galaxy has built up $X \%$ of its total stellar mass. 
    * *potential issue*: may not be a good proxy for the precise quantity we're looking for.
* In the *most recent* star formation event, find the earliest time index at which the mass in each bin exceeds some minimum value, e.g., 1e7 solar masses or 0.01% of the total stellar mass.
    * *potential issue*: bins are not linearly spaced in age - how to deal with this?
* In the *most recent* star formation event, find the earliest time index at which the SFR exceeds some minimum value, e.g. $1 \rm \, M_\odot \, yr^{-1}$.
    * *potential issue*: this may not represent the actual SFR in the galaxy, if the new stars were obtained via a merger, for instance. 


In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [3]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import load_sfh, create_mock_spectrum, calculate_mw_age
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [4]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
sigma_star_kms = 250
z = 0.01

# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)


In [6]:
# Definition 1: time index at which most recent star formation event drops to 0
def compute_sb_zero_age(sfh_mw):
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw
        
    first_nonzero_idx = np.argwhere(sfh_mw_1D > 0)[0][0]
    if np.any(sfh_mw_1D[first_nonzero_idx:] == 0):
        first_zero_idx = np.argwhere(sfh_mw_1D[first_nonzero_idx:] == 0)[0][0] + first_nonzero_idx
        return ages[first_zero_idx], first_zero_idx
    else:
        return np.nan, np.nan

In [7]:
# Definition 2: Mass weighted age below some age threshold
def compute_mw_age(sfh_mw, 
                   age_thresh=1e9):
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw

    # Find the index of the threshold age in the template age array
    age_thresh_idx = np.nanargmin(np.abs(ages - age_thresh))
    
    # Compute the mass-weighted age 
    log_age_mw = np.nansum(sfh_mw_1D[:age_thresh_idx] * np.log10(ages[:age_thresh_idx])) / np.nansum(sfh_mw_1D[:age_thresh_idx])
    
    # Compute the corresponding index in the array (useful for plotting)
    log_age_mw_idx = (log_age_mw - np.log10(ages[0])) / (np.log10(ages[1]) - np.log10(ages[0]))
    
    return 10**log_age_mw, log_age_mw_idx


In [8]:
# Definition 3: Counting backwards from $t = 0$, the time index at which the galaxy has built up $X \%$ of its total stellar mass. 
def compute_first_massive_bin(sfh_mw,
                              mass_frac_thresh=0.01):
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw

    M_tot = np.nansum(sfh_mw_1D) 
    age_idx = np.argwhere(sfh_mw_1D / M_tot > mass_frac_thresh)[0]
        
    return ages[age_idx], age_idx


In [9]:
# Definition 4: In the *most recent* star formation event, find the earliest time index at which the mass in each bin exceeds 
# some minimum value, e.g., 1e7 solar masses or 0.01% of the total stellar mass.
def compute_cumulative_mass_frac(sfh_mw,
                                 mass_frac_thresh=0.01):
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw

    M_tot = np.nansum(sfh_mw_1D) 
    cumsum_M_frac = np.cumsum(sfh_mw_1D) / M_tot
    age_idx = np.nanargmin(np.abs(cumsum_M_frac - mass_frac_thresh))
    
    return ages[age_idx], age_idx


In [14]:
def compute_sfr_thresh_age(sfh_mw,
                           sfr_thresh=1.0):
    
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw

    # Compute the bin edges and widths so that we can compute the mean SFR in each bin
    bin_widths = np.zeros(len(ages))
    bin_edges = np.zeros(len(ages) + 1)
    for aa in range(1, len(ages)):
        bin_edges[aa] = 10**(0.5 * (np.log10(ages[aa - 1]) + np.log10(ages[aa])) )
    delta_log_age = np.diff(np.log10(ages))[0]
    age_0 = 10**(np.log10(ages[0]) - delta_log_age)
    age_last = 10**(np.log10(ages[-1]) + delta_log_age)
    bin_edges[0] = 10**(0.5 * (np.log10(age_0) + np.log10(ages[0])) )
    bin_edges[-1] = 10**(0.5 * (np.log10(ages[-1]) + np.log10(age_last)))
    bin_sizes = np.diff(bin_edges)
    
    # Compute the mean SFR in each bin
    sfr_avg = sfh_mw_1D / bin_sizes
    
    # Find the first index where the SFR exceed a certain value
    if np.any(sfr_avg > sfr_thresh):
        age_idx = np.argwhere(sfr_avg > sfr_thresh)[0][0]
        return ages[age_idx], age_idx
    else:
        return np.nan, np.nan
    

In [15]:
sfh = load_sfh(11, plotit=True)

# Def. 1
print(f"compute_sb_zero_age(): {compute_sb_zero_age(sfh)[0] / 1e6:.2f} Myr")

# Def. 2
print(f"compute_mw_age(): {compute_mw_age(sfh, age_thresh=1e8)[0] / 1e6:.2f} Myr")

# Def. 3
print(f"compute_first_massive_bin(): {compute_first_massive_bin(sfh, mass_frac_thresh=0.001)[0] / 1e6:.2f} Myr")

# Def. 4
print(f"compute_cumulative_mass_frac(): {compute_cumulative_mass_frac(sfh, mass_frac_thresh=0.001)[0] / 1e6:.2f} Myr")

# Def. 5
print(f"{compute_sfr_thresh_age(sfh, sfr_thresh=0.5)[0] / 1e6:.2f} Myr")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100.00 Myr
89.13 Myr
501.19 Myr
89.12 Myr


In [145]:
# Test: compute_cumulative_mass_frac()
sfh = load_sfh(11, plotit=True)
sfh_1D = np.nansum(sfh, axis=0)
M_tot = np.nansum(sfh)

mass_frac_thresh = 1e-1
age_idx_4 = compute_cumulative_mass_frac(sfh, mass_frac_thresh)[1]
age_idx_3 = compute_first_massive_bin(sfh, mass_frac_thresh)[1]

# Plot 
fig, ax = plt.subplots(figsize=(10, 4))
fig.subplots_adjust(bottom=0.3, top=0.9)

ax.step(x=range(N_ages), y=sfh_1D / M_tot, where="mid", label="1D SFH")
ax.step(x=range(N_ages), y=np.cumsum(sfh_1D) / M_tot, where="mid", label="Cumulative mass")
ax.axvline(age_idx_4, color="red", label="Definition 4")
ax.axvline(age_idx_3, color="blue", label="Definition 3")
ax.axhline(mass_frac_thresh, color="grey", label="Mass fraction threshold")
ax.set_yscale("log")
ax.set_xticks(range(N_ages))
ax.set_xticklabels(ages / 1e6, rotation="vertical", fontsize="x-small")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)

# Plot the mean SFR in each bin, too
# Compute the bin edges and widths so that we can compute the mean SFR in each bin
bin_widths = np.zeros(len(ages))
bin_edges = np.zeros(len(ages) + 1)
for aa in range(1, len(ages)):
    bin_edges[aa] = 10**(0.5 * (np.log10(ages[aa - 1]) + np.log10(ages[aa])) )
delta_log_age = np.diff(np.log10(ages))[0]
age_0 = 10**(np.log10(ages[0]) - delta_log_age)
age_last = 10**(np.log10(ages[-1]) + delta_log_age)
bin_edges[0] = 10**(0.5 * (np.log10(age_0) + np.log10(ages[0])) )
bin_edges[-1] = 10**(0.5 * (np.log10(ages[-1]) + np.log10(age_last)))
bin_sizes = np.diff(bin_edges)

# Plot the mean SFR in each bin
sfr_avg = sfh_1D / bin_sizes

fig, ax = plt.subplots(figsize=(10, 4))
fig.subplots_adjust(bottom=0.3, top=0.9)
ax.step(x=range(N_ages), y=sfr_avg, where="mid", label="Average SFR")
ax.set_yscale("log")
ax.set_xticks(range(N_ages))
ax.set_xticklabels(ages / 1e6, rotation="vertical", fontsize="x-small")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …